<a href="https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/LiLT/Fine_tune_LiltForTokenClassification_on_FUNSD_(nielsr_funsd).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set-up environment

In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git

rm: cannot remove 'transformers': No such file or directory
Cloning into 'transformers'...
remote: Enumerating objects: 142169, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 142169 (delta 52), reused 77 (delta 49), pack-reused 142078
Receiving objects: 100% (142169/142169), 118.37 MiB | 16.63 MiB/s, done.
Resolving deltas: 100% (104961/104961), done.
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163 kB 27.7 MB/s

In [2]:
!pip install -q datasets

     |████████████████████████████████| 432 kB 23.8 MB/s 
     |████████████████████████████████| 115 kB 74.0 MB/s 
     |████████████████████████████████| 212 kB 75.1 MB/s 
     |████████████████████████████████| 127 kB 65.3 MB/s 


## Load dataset

In [3]:
from datasets import load_dataset

dataset = load_dataset("nielsr/funsd")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset funsd downloaded and prepared to /root/.cache/huggingface/datasets/nielsr___funsd/funsd/1.0.0/8b0472b536a2dcb975d59a4fb9d6fea4e6a1abe260b7fed6f75301e168cbe595. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'words', 'bboxes', 'ner_tags', 'image_path'],
        num_rows: 149
    })
    test: Dataset({
        features: ['id', 'words', 'bboxes', 'ner_tags', 'image_path'],
        num_rows: 50
    })
})

In [5]:
dataset["train"].features

{'id': Value(dtype='string', id=None),
 'words': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'bboxes': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-HEADER', 'I-HEADER', 'B-QUESTION', 'I-QUESTION', 'B-ANSWER', 'I-ANSWER'], id=None), length=-1, id=None),
 'image_path': Value(dtype='string', id=None)}

In [6]:
labels = dataset["train"].features['ner_tags'].feature.names
id2label = {id:label for id, label in enumerate(labels)}
label2id = {label:id for id, label in enumerate(labels)}
print(id2label)

{0: 'O', 1: 'B-HEADER', 2: 'I-HEADER', 3: 'B-QUESTION', 4: 'I-QUESTION', 5: 'B-ANSWER', 6: 'I-ANSWER'}


In [7]:
example = dataset["train"][0]
print(example["words"])
print(example["bboxes"])
print(example["ner_tags"])

['R&D', ':', 'Suggestion:', 'Date:', 'Licensee', 'Yes', 'No', '597005708', 'R&D', 'QUALITY', 'IMPROVEMENT', 'SUGGESTION/', 'SOLUTION', 'FORM', 'Name', '/', 'Phone', 'Ext.', ':', 'M.', 'Hamann', 'P.', 'Harper,', 'P.', 'Martinez', '9/', '3/', '92', 'R&D', 'Group:', 'J.', 'S.', 'Wigand', 'Supervisor', '/', 'Manager', 'Discontinue', 'coal', 'retention', 'analyses', 'on', 'licensee', 'submitted', 'product', 'samples', '(Note', ':', 'Coal', 'Retention', 'testing', 'is', 'not', 'performed', 'by', 'most', 'licensees.', 'Other', 'B&W', 'physical', 'measurements', 'as', 'ends', 'stability', 'and', 'inspection', 'for', 'soft', 'spots', 'in', 'ciparettes', 'are', 'thought', 'to', 'be', 'sufficient', 'measures', 'to', 'assure', 'cigarette', 'physical', 'integrity.', 'The', 'proposed', 'action', 'will', 'increase', 'laboratory', 'productivity', '.', ')', 'Suggested', 'Solutions', '(s)', ':', 'Delete', 'coal', 'retention', 'from', 'the', 'list', 'of', 'standard', 'analyses', 'performed', 'on', 'licen

## Transform dataset

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/layoutlmv3-base")

Downloading:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [9]:
def prepare_examples(batch):
  encoding = tokenizer(batch["words"],
                        boxes=batch["bboxes"],
                        word_labels=batch["ner_tags"],
                        padding="max_length",
                        max_length=128,
                        truncation=True,
                        return_tensors="pt")
  
  return encoding

dataset.set_transform(prepare_examples)

In [10]:
example = dataset["train"][0]
print(example.keys())

dict_keys(['input_ids', 'attention_mask', 'bbox', 'labels'])


In [11]:
tokenizer.decode(example["input_ids"])

'<s> R&D : Suggestion: Date: Licensee Yes No 597005708 R&D QUALITY IMPROVEMENT SUGGESTION/ SOLUTION FORM Name / Phone Ext. : M. Hamann P. Harper, P. Martinez 9/ 3/ 92 R&D Group: J. S. Wigand Supervisor / Manager Discontinue coal retention analyses on licensee submitted product samples (Note : Coal Retention testing is not performed by most licensees. Other B&W physical measurements as ends stability and inspection for soft spots in ciparettes are thought to be sufficient measures to assure cigarette</s>'

In [12]:
for id, box, label in zip(example["input_ids"].tolist(),
                          example["bbox"].tolist(),
                          example["labels"].tolist()):
  if label != -100:
    print(tokenizer.decode([id]), box, id2label[label])
  else:
    print(tokenizer.decode([id]), box, label)

<s> [0, 0, 0, 0] -100
 R [383, 91, 493, 175] O
& [383, 91, 493, 175] -100
D [383, 91, 493, 175] -100
 : [287, 316, 295, 327] B-QUESTION
 Suggest [124, 355, 221, 370] B-QUESTION
ion [124, 355, 221, 370] -100
: [124, 355, 221, 370] -100
 Date [632, 268, 679, 282] B-QUESTION
: [632, 268, 679, 282] -100
 License [670, 309, 748, 323] B-ANSWER
e [670, 309, 748, 323] -100
 Yes [604, 605, 633, 619] B-QUESTION
 No [715, 603, 738, 617] B-QUESTION
 5 [688, 904, 841, 926] O
97 [688, 904, 841, 926] -100
005 [688, 904, 841, 926] -100
708 [688, 904, 841, 926] -100
 R [337, 203, 366, 214] B-HEADER
& [337, 203, 366, 214] -100
D [337, 203, 366, 214] -100
 QU [374, 203, 438, 216] I-HEADER
AL [374, 203, 438, 216] -100
ITY [374, 203, 438, 216] -100
 IM [447, 201, 548, 211] I-HEADER
PROV [447, 201, 548, 211] -100
EMENT [447, 201, 548, 211] -100
 S [335, 215, 425, 229] I-HEADER
UG [335, 215, 425, 229] -100
G [335, 215, 425, 229] -100
EST [335, 215, 425, 229] -100
ION [335, 215, 425, 229] -100
/ [335, 215, 42

## Create PyTorch Dataloaders

In [13]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset["train"], batch_size=2, shuffle=True)
test_dataloader = DataLoader(dataset["test"], batch_size=2, shuffle=True)

In [14]:
batch = next(iter(train_dataloader))
for k,v in batch.items():
  print(k,v.shape)

input_ids torch.Size([2, 128])
attention_mask torch.Size([2, 128])
bbox torch.Size([2, 128, 4])
labels torch.Size([2, 128])


## Load model

In [15]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("nielsr/lilt-roberta-en-base", id2label=id2label, label2id=label2id)

Downloading:   0%|          | 0.00/697 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Some weights of LiltForTokenClassification were not initialized from the model checkpoint at nielsr/lilt-roberta-en-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Train!

In [16]:
!pip install -q evaluate seqeval

     |████████████████████████████████| 69 kB 7.2 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 


In [17]:
import evaluate
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Metric
metric = evaluate.load("seqeval")

def get_labels(predictions, references):
    # Transform predictions and references tensors to numpy arrays
    if device.type == "cpu":
        y_pred = predictions.detach().clone().numpy()
        y_true = references.detach().clone().numpy()
    else:
        y_pred = predictions.detach().cpu().clone().numpy()
        y_true = references.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens)
    true_predictions = [
        [labels[p] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    true_labels = [
        [labels[l] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    return true_predictions, true_labels

In [18]:
from torch.optim import AdamW
from tqdm.auto import tqdm

optimizer = AdamW(model.parameters(), lr=5e-5)

model.to(device)

for epoch in range(50):
  print("Epoch:", epoch+1)
  for idx, batch in enumerate(tqdm(train_dataloader)):
      # move batch to device
      batch = {k:v.to(device) for k,v in batch.items()}
      outputs = model(**batch)

      predictions = outputs.logits.argmax(-1)
      true_predictions, true_labels = get_labels(predictions, batch["labels"])
      metric.add_batch(references=true_labels, predictions=true_predictions)

      loss = outputs.loss

      if idx % 100 == 0:
        print("Loss:", loss.item())
        results = metric.compute()
        print("Overall f1:", results["overall_f1"])
        print("Overall precision:", results["overall_f1"])
        print("Overall recall:", results["overall_recall"])

      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

Epoch: 1


  0%|          | 0/75 [00:00<?, ?it/s]

Loss: 2.00651216506958
Overall f1: 0.032
Overall precision: 0.032
Overall recall: 0.037037037037037035
Epoch: 2


  0%|          | 0/75 [00:00<?, ?it/s]

Loss: 1.1887439489364624
Overall f1: 0.48512585812356973
Overall precision: 0.48512585812356973
Overall recall: 0.5553892215568862
Epoch: 3


...

Loss: 0.0001653546787565574
Overall f1: 1.0
Overall precision: 1.0
Overall recall: 1.0
Epoch: 49


  0%|          | 0/75 [00:00<?, ?it/s]

Loss: 0.00024371480685658753
Overall f1: 1.0
Overall precision: 1.0
Overall recall: 1.0
Epoch: 50


  0%|          | 0/75 [00:00<?, ?it/s]

Loss: 0.00015494016406591982
Overall f1: 1.0
Overall precision: 1.0
Overall recall: 1.0


## Evaluate

In [19]:
from tqdm.auto import tqdm

eval_metric = evaluate.load("seqeval")

for idx, batch in enumerate(tqdm(test_dataloader)):
    # move batch to device
    batch = {k:v.to(device) for k,v in batch.items()}
    with torch.no_grad():
      outputs = model(**batch)

    predictions = outputs.logits.argmax(-1)
    true_predictions, true_labels = get_labels(predictions, batch["labels"])
    eval_metric.add_batch(references=true_labels, predictions=true_predictions)

  0%|          | 0/25 [00:00<?, ?it/s]

In [20]:
results = eval_metric.compute()
results["overall_f1"]

0.7719969395562356

In [21]:
results["overall_accuracy"]

0.7827023699599877